In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

# ---------------------------
# 1. Data Preparation
# ---------------------------

# Load your stock data and SP500 data
stocks_df = pd.read_excel('Stock_Features_Complete.xlsx', parse_dates=['Date'])
sp500_df = pd.read_excel('sp500_Cleaned_Processed.xlsx', parse_dates=['Date'])

# Filter last 5 years of data
end_date = stocks_df['Date'].max()
start_date = end_date - pd.DateOffset(years=5)
stocks_df = stocks_df[(stocks_df['Date'] >= start_date) & (stocks_df['Date'] <= end_date)]

stocks_df.head()

,Date,Ticker,Closing_Price,Simple_Return,Log_Return,Rolling_20D_Volatility,SMA_50,SMA_200,EMA_12,EMA_26,MACD,RSI,SMA_20,Upper_BB,Lower_BB
0,2020-03-31,A UN Equity,71.62,NaN,NaN,NaN,NaN,NaN,71.620000,71.620000,0.000000,NaN,NaN,NaN,NaN
1,2020-04-30,A UN Equity,76.66,0.070371,0.068006,NaN,NaN,NaN,72.395385,71.993333,0.402051,NaN,NaN,NaN,NaN
2,2020-05-29,A UN Equity,88.14,0.149752,0.139546,NaN,NaN,NaN,74.817633,73.189383,1.628250,NaN,NaN,NaN,NaN
3,2020-06-30,A UN Equity,88.37,0.002609,0.002606,NaN,NaN,NaN,76.902613,74.313873,2.588740,NaN,NaN,NaN,NaN
4,2020-07-31,A UN Equity,96.33,0.090076,0.086247,NaN,84.224,84.224,79.891441,75.944697,3.946744,100.0,84.224,NaN,NaN


In [10]:
# Split into training and backtesting periods
cutoff_date = end_date - pd.DateOffset(months=5)
train = stocks_df[stocks_df['Date'] < cutoff_date]
test = stocks_df[stocks_df['Date'] >= cutoff_date]
print(cutoff_date)

2024-09-28 00:00:00


In [11]:
def arima_forecast(ticker_data, periods=5):
    """Fit ARIMA and forecast returns"""
    model = ARIMA(ticker_data['Closing_Price'], order=(1,1,1))  # Basic parameters
    fitted_model = model.fit()
    forecast = fitted_model.forecast(steps=periods)
    return forecast

In [12]:
forecasts = {}

# Process each ticker
for ticker in stocks_df['Ticker'].unique():
    ticker_data = train[train['Ticker'] == ticker].sort_values('Date')
    if len(ticker_data) > 10:  # Ensure sufficient data
        try:
            forecast = arima_forecast(ticker_data)
            last_price = ticker_data['Closing_Price'].iloc[-1]
            expected_return = (forecast.iloc[-1] - last_price)/last_price
            forecasts[ticker] = expected_return
        except:
            continue

/opt/anaconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/anaconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next v

In [13]:
top_50 = pd.Series(forecasts).nlargest(50).index.tolist()
print(top_50)

['IRM UN Equity', 'NVDA UW Equity', 'TRGP UN Equity', 'PGR UN Equity', 'AVGO UW Equity', 'TT UN Equity', 'LII UN Equity', 'HWM UN Equity', 'KKR UN Equity', 'GRMN UN Equity', 'CTAS UW Equity', 'PYPL UW Equity', 'GE UN Equity', 'TDG UN Equity', 'KLAC UW Equity', 'VRSN UW Equity', 'CPT UN Equity', 'VST UN Equity', 'LUV UN Equity', 'MCK UN Equity', 'JPM UN Equity', 'ETN UN Equity', 'NVR UN Equity', 'MSI UN Equity', 'BA UN Equity', 'SWK UN Equity', 'AFL UN Equity', 'HIG UN Equity', 'NOW UN Equity', 'GS UN Equity', 'ERIE UW Equity', 'AXP UN Equity', 'MHK UN Equity', 'PNR UN Equity', 'L UN Equity', 'MSFT UW Equity', 'GD UN Equity', 'WBA UW Equity', 'CBOE UF Equity', 'MCHP UW Equity', 'NFLX UW Equity', 'MGM UN Equity', 'BEN UN Equity', 'GNRC UN Equity', 'NUE UN Equity', 'CRL UN Equity', 'NWSA UW Equity', 'SJM UN Equity', 'CDW UW Equity', 'INTC UW Equity']


In [23]:
pip install pandas numpy matplotlib seaborn pypfopt

ERROR: Could not find a version that satisfies the requirement pypfopt (from versions: none)
ERROR: No matching distribution found for pypfopt
Note: you may need to restart the kernel to use updated packages.


In [25]:
print(train)

            Date         Ticker  Closing_Price  Simple_Return  Log_Return  \
0     2020-03-31    A UN Equity          71.62            NaN         NaN   
1     2020-04-30    A UN Equity          76.66       0.070371    0.068006   
2     2020-05-29    A UN Equity          88.14       0.149752    0.139546   
3     2020-06-30    A UN Equity          88.37       0.002609    0.002606   
4     2020-07-31    A UN Equity          96.33       0.090076    0.086247   
...          ...            ...            ...            ...         ...   
28729 2024-04-30  ZTS UN Equity         159.24      -0.058921   -0.060728   
28730 2024-05-31  ZTS UN Equity         169.56       0.064808    0.062794   
28731 2024-06-28  ZTS UN Equity         173.36       0.022411    0.022164   
28732 2024-07-31  ZTS UN Equity         180.04       0.038533    0.037809   
28733 2024-08-30  ZTS UN Equity         183.49       0.019162    0.018981   

       Rolling_20D_Volatility    SMA_50     SMA_200      EMA_12      EMA_26

In [30]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from pypfopt import EfficientFrontier, expected_returns, risk_models, objective_functions
import matplotlib.pyplot as plt

# ---------------------------
# 1. Data Preparation & Cleaning
# ---------------------------

# Read data and convert to proper format
df = pd.read_excel('Stock_Features_Complete.xlsx', parse_dates=['Date'])

# Pivot to wide format with proper structure
wide_df = df.pivot(index='Date', columns='Ticker', values='Closing_Price')

# Filter last 5 years of data
end_date = wide_df.index.max()
cutoff_date = end_date - pd.DateOffset(months=5)
train = wide_df.loc[wide_df.index <= cutoff_date]
test = wide_df.loc[wide_df.index > cutoff_date]

# ---------------------------
# 2. ARIMA Forecasting
# ---------------------------

def arima_forecast(series, periods=5):
    """Robust ARIMA forecasting with error handling"""
    try:
        # Check data adequacy
        if len(series.dropna()) < 20:
            return np.nan
            
        model = ARIMA(series.dropna(), order=(1,1,1))
        results = model.fit()
        forecast = results.forecast(steps=periods)
        return (forecast.iloc[-1] - series.iloc[-1]) / series.iloc[-1]
    except Exception as e:
        print(f"Failed on {series.name}: {str(e)}")
        return np.nan

# Calculate forecasts for all tickers
forecasts = {}
for ticker in train.columns:
    ticker_series = train[ticker].dropna()
    if len(ticker_series) > 20:  # Minimum 20 observations
        ret = arima_forecast(ticker_series)
        if not np.isnan(ret):
            forecasts[ticker] = ret

# Select top 50 valid tickers
top_50 = pd.Series(forecasts).nlargest(50).index.tolist()

# ---------------------------
# 3. Portfolio Optimization
# ---------------------------

# Prepare returns data
returns_df = train[top_50].pct_change().dropna(how='all', axis=1)

# Handle remaining missing values
returns_df = returns_df.ffill().bfill().dropna()

# Calculate inputs
mu = expected_returns.mean_historical_return(returns_df)
S = risk_models.sample_cov(returns_df)

# Ensure positive semi-definite matrix
def make_psd(matrix):
    """Fix non-positive semi-definite covariance matrix"""
    min_eig = np.min(np.real(np.linalg.eigvals(matrix)))
    if min_eig < 0:
        matrix -= 10*min_eig * np.eye(*matrix.shape)
    return matrix

S = make_psd(S)

# Optimize portfolio
ef = EfficientFrontier(mu, S)
ef.add_objective(objective_functions.L2_reg, gamma=0.1)

# Do this:
try:
    ef1 = EfficientFrontier(mu, S)
    ef1.max_sharpe()
except:
    ef2 = EfficientFrontier(mu, S)  # New instance
    ef2.min_volatility()

weights = ef.clean_weights(threshold=0.01)  # Minimum 1% allocation

# ---------------------------
# 4. Backtesting & Comparison
# ---------------------------

# Prepare test data
test_returns = test[top_50].pct_change()

# Align dates between portfolio and SP500
sp500 = pd.read_excel('sp500_Cleaned_Processed.xlsx', 
                     parse_dates=['Date']).set_index('Date')
common_dates = test_returns.index.intersection(sp500.index)

# Calculate portfolio returns
portfolio_returns = test_returns.loc[common_dates].dot(pd.Series(weights))

# SP500 returns
sp500_returns = sp500.loc[common_dates, 'Simple_Return']

# ---------------------------
# 5. Visualizations
# ---------------------------

# Cumulative returns comparison
plt.figure(figsize=(12,6))
(1 + portfolio_returns).cumprod().plot(label='Optimized Portfolio')
(1 + sp500_returns).cumprod().plot(label='SP500', linestyle='--')
plt.title('Performance Comparison: Portfolio vs SP500')
plt.ylabel('Growth of $1 Investment')
plt.legend()
plt.grid(True)
plt.show()

# Portfolio weights distribution
plt.figure(figsize=(10,6))
pd.Series(weights).sort_values().plot.barh()
plt.title('Portfolio Weight Allocation')
plt.xlabel('Weight Percentage')
plt.show()

# Efficient frontier visualization
fig, ax = plt.subplots()
ef = EfficientFrontier(mu, S)
ef.plot_efficient_frontier(ax=ax, show_assets=False)
plt.title('Efficient Frontier')
plt.show()

/opt/anaconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency BME will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency BME will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency BME will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency BME will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency BME will b

ERROR in LDL_factor: Error in KKT matrix LDL factorization when computing the nonzero elements. The problem seems to be non-convex
ERROR in osqp_setup: KKT matrix factorization.
The problem seems to be non-convex.


/opt/anaconda3/lib/python3.12/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(


InstantiationError: Adding constraints to an already solved problem might have unintended consequences. A new instance should be created for the new set of constraints.